In [2]:
class Pair:
    def __init__(self, x, y):
        self.x  = x
        self.y = y
    
    #It is standard practice for the output of __repr__() to produce text such that eval(repr(x)) == x
    def __repr__(self): # returns the code represenation of an instance
        return 'Pair({0.x!r}, {0.y!r})'.format(self)
    
    def __str__(self): # returns the str representation of an instance
        return '({0.x!s}, {0.y!s})'.format(self)
    
p = Pair(10,20)

print(p) # p.__str__()
p # p.__repr__()


(10, 20)


Pair(10, 20)

In [3]:
### Customizing string formatting

_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
    }

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [4]:
d = Date(2012, 12, 21)

print(format(d))

print(format(d, 'mdy'))

print('The date is {:ymd}'.format(d))

print('The date is {:mdy}'.format(d))


2012-12-21
12/21/2012
The date is 2012-12-21
The date is 12/21/2012


###  Making Objects Support the Context-Management Protocol

In [7]:
"""
In order to make an object compatible with the with statement, you need to implement __enter__() and __exit__() methods.
"""

from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None

    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

In [9]:
from functools import partial

conn = LazyConnection(('www.python.org', 80))

with conn as s:
    # conn.__enter__() starts
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))

### Allowing multiple connections instead

In [10]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.connections = []

    def __enter__(self):
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connections.append(sock)
        return sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.connections.pop().close()

# Example use
from functools import partial

conn = LazyConnection(('www.python.org', 80))
with conn as s1:
     print('socker 1')
     with conn as s2:
         print('socket 2')

socker 1
socket 2


### Saving memory creating large number of instances

In [ ]:
""" We can reduce the memory footprint of instances by adding the __slots__ attribute to the class def """

"""
 A side effect of using slots is that it is no longer possible to add 
 new attributes to instances—you are restricted to only those attribute names 
 listed in the __slots__ specifier.
"""
class Date:
    
    __slots__ = ['year', 'month', 'day']
    
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

In [ ]:
#### Class variable naming

"""
class B
_hello -> private attb of a class B, which we don't expect to be inherited
__world -> use double underscore, if B will be inherited as it will be manged to _B_world and _C_world respectively - hence remain private

class C(B)
__world -> _C_world

to indicate a variable not to confuse with keywords, use trailing underscore
eg: lambda_
"""

### Creating Managed Attributes

In [12]:
"""
Adding extra processing to the getting and setting of an instance attribute
- we can customize the access to an attribute by defining it as property
"""

class Person:

    def __init__(self, first_name):
        self.first_name = first_name

    # getter function
    @property
    def first_name(self):
        return self._first_name
    
    # setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
    
    # deleter function 
    @first_name.deleter
    def first_name(self):
        raise AttributeError('cannot delete attribute')


In [13]:
# Testing

p = Person('riaz')
p.first_name

'riaz'

In [14]:
p.first_name = 19

TypeError: Expected a string

In [15]:
del p.first_name

AttributeError: cannot delete attribute

### Defining properties by existing get and set methods

In [18]:
class Person:
    def __init__(self, first_name):
        self.set_first_name(first_name)
    
    # getter
    def get_first_name(self):
        return self._first_name
    
    # setter
    def set_first_name(self, first_name):
        if not isinstance(first_name, str):
            raise ValueError('Expected str value')
        self._first_name = first_name

    # deleter
    def del_first_name(self):
        raise AttributeError('Cannot delete attribute')


    # making a property from existing get,set methods
    first_name = property(get_first_name, set_first_name, del_first_name)
    

In [20]:
p = Person("riaz")
p.__dir__()

['_first_name',
 '__module__',
 '__init__',
 'get_first_name',
 'set_first_name',
 'del_first_name',
 'first_name',
 '__dict__',
 '__weakref__',
 '__doc__',
 '__new__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [22]:
print(Person.first_name.fget)
print(Person.first_name.fset)
print(Person.first_name.fdel)

<function __main__.Person.get_first_name(self)>

### Using properties to define computer attributes
# example: area and perimeter of a circle

In [23]:
import math

class Circle:
    """
    Note: these properties are computed on demand and doesn't have brackets.
    """
    def __init__(self, radius):
        self.radius = radius
    
    @property
    def area(self):
        return math.pi * self.radius ** 2
    @property
    def perimeter(self):
        return 2 * math.pi * self.radius


In [24]:
c = Circle(4.0)

print(c.radius)

print(c.area)

print(c.perimeter)


4.0
50.26548245743669
25.132741228718345


In [25]:
c.radius = 20

print(c.area)

print(c.perimeter)

1256.6370614359173
125.66370614359172


### Calling a method on a parent class

In [39]:
class A:
    def spam(self):
        print("This is spam from class A")

class B(A):
    def spam(self):
        super().spam()
        print("This is spam from class B")


b = B()

b.spam()

This is spam from class A
This is spam from class B


In [ ]:
# common use of the super method is to invoke the init methid

class A:
    def __init__(self):
        self.x = 0

class B(A):
    # this init method of the subclass is trying to add a new member
    def __init__(self):
        super.__init__()
        self.y = 1

### Multiple Inheritence and super

In [42]:
class Base:
    def __init__(self):
        print('Base.__init__')
    
class A(Base):
    def __init__(self):
        Base.__init__(self) # directly calling the parent constructor
        print("A.__init__")

class B(Base):
    def __init__(self):
        Base.__init__(self) # directly calling the parent constructor
        print("B.__init__")

class C(A, B):
    def __init__(self):
        A.__init__(self) # directly calling the parent constructor
        B.__init__(self) # directly calling the parent constructor
        print("C.__init__")

C() # Base is called twice

Base.__init__
A.__init__
Base.__init__
B.__init__
C.__init__


In [44]:
# Alternative to above

class Base:
    def __init__(self):
        print('Base.__init__')
    
class A(Base):
    def __init__(self):
        super().__init__() 
        print("A.__init__")

class B(Base):
    def __init__(self):
        super().__init__()
        print("B.__init__")

class C(A, B):
    def __init__(self):
        super().__init__()         
        print("C.__init__")

C() # Base is called twice

Base.__init__
B.__init__
A.__init__
C.__init__


In [45]:
# Python will invoke a method resolution order - mro
C.__mro__

(__main__.C, __main__.A, __main__.B, __main__.Base, object)

In [ ]:
### Extending the property of a subclass

In [50]:
class Person:
    def __init__(self, name):
        self.name = name
    
    #Getter
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, name):
        if not isinstance(name, str):
            raise ValueError('Expecting a string type')
        self._name = name
    
    @name.deleter
    def name(self):
        raise AttributeError('Cannot delete attribute')


In [64]:
# here is the example of a class that extends the name type of Person

class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name
    
    @name.setter
    def name(self, value):
        print('Setting name to', value)
        # not sure what this line does
        super(SubPerson, SubPerson).name.__set__(self, value)
    
    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)


In [65]:
s = SubPerson('Riaz')

print(s.name)

s.name = 'Munshi'

try:
    s.name = 42 # this will throw a error
except ValueError as ve:
    print(ve)

Setting name to Riaz
Getting name
Riaz
Setting name to Munshi
Setting name to 42
Expecting a string type


In [69]:
## If extending only on of the methods of the property
class SubPerson(Person):
    @Person.name.getter
    def name(self):
        print('Getting the name')
        return super().name

In [70]:
s = SubPerson('Riaz')

print(s.name)

Getting the name
Riaz


In [ ]:
#### Creating a new Type by defining it as a Descriptor class

In [72]:
"""
A descriptor has the following methods
get, set and delete
"""

class Integer:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected an int')
        instance.__dict__[self.name] = value
    
    def __delete__(self, instance):
        del instance.__dict__[self.name]

In [73]:
# Note: descriptors only work with class variables
class Point:
    x = Integer('x') # Note: this is a class variable
    y = Integer('y')
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [75]:
p = Point(2, 3)

print(p.x, p.y)

try:
    p.x = 2.3 # trying to put a float
except TypeError as e:
    print(e)

2 3
Expected an int


In [76]:
p = Point(2,3)

print(p.x) # x access as instance - from dict in get

print(Point.x) # x accessed as cls variable

2


: 